In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm
import requests
from geopy.geocoders import GoogleV3

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy import geocoders 
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook
from urllib.parse import urlencode

Populating the interactive namespace from numpy and matplotlib
time: 2.44 s (started: 2021-12-17 22:53:25 -06:00)


In [2]:
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")

time: 0 ns (started: 2021-12-17 22:53:28 -06:00)


In [ ]:
feblistings = pd.read_csv('../data/feb2021listings.csv')
feblistings.shape

In [ ]:
feblistings.info()

In [ ]:
feblistings = feblistings.drop(['neighbourhood'], axis=1)

In [ ]:
feblistings.dropna(how='all', axis=1, inplace=True)

In [ ]:
feblistings.head()

In [ ]:
feblistings['latitude'].value_counts()

In [ ]:
feblistings['longitude'].value_counts()

In [ ]:
feblistings = pd.read_csv('../data/feb2021listings.csv')
feblistings.shape

In [ ]:
#march month not available in data

In [ ]:
Aprillistings = pd.read_csv('../data/April2021listings.csv')
Aprillistings.shape

In [ ]:
Aprillistings['longitude'].value_counts()

In [ ]:
Junelistings = pd.read_csv('../data/June2021listings.csv')
Junelistings.shape

In [ ]:
Junelistings['longitude'].value_counts()

In [ ]:
Julylistings = pd.read_csv('../data/July2021listings.csv')
Julylistings.shape

In [ ]:
Julylistings['longitude'].value_counts()

In [ ]:
Augustlistings = pd.read_csv('../data/August2021listings.csv')
Augustlistings.shape

In [ ]:
Augustlistings['longitude'].value_counts()

In [ ]:
Septemberlistings = pd.read_csv('../data/September2021listings.csv')
Septemberlistings.shape

In [ ]:
Septemberlistings['longitude'].value_counts()

In [ ]:
Octoberlistings = pd.read_csv('../data/October2021listings.csv')
Octoberlistings.shape

In [ ]:
Octoberlistings['longitude'].value_counts()

In [ ]:
Octoberlistings['latitude'].value_counts()

In [ ]:
#i cannot find the nan,nan values in any of the months - maybe it is duplicates from when the information was merged
# i will go ahead and drop all nan rows in my data and attempt to convert all addresses again
# if that does not work, i will need to convert addresses individually by month

In [3]:
all_filenames = [i for i in glob.glob('../data/monthlistings/*.{}'.format('csv'))]

time: 0 ns (started: 2021-12-17 22:53:34 -06:00)


In [4]:
print(all_filenames)

['../data/monthlistings\\April2021listings.csv', '../data/monthlistings\\August2021listings.csv', '../data/monthlistings\\December2020listings.csv', '../data/monthlistings\\feb2021listings.csv', '../data/monthlistings\\January2021listings.csv', '../data/monthlistings\\July2021listings.csv', '../data/monthlistings\\June2021listings.csv', '../data/monthlistings\\October2021listings.csv', '../data/monthlistings\\September2021listings.csv']
time: 0 ns (started: 2021-12-17 22:53:38 -06:00)


In [5]:
combined_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 250 ms (started: 2021-12-17 22:53:39 -06:00)


In [6]:
combined_months.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6422,Nashville Charm,12172,Michele,NaN,District 6,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,NaN
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,NaN,District 6,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,NaN
2,39870,Close to Vanderbilt 2,171184,Evelyn,NaN,District 18,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,NaN
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,NaN,District 12,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,NaN
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,NaN,District 12,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,NaN


time: 31 ms (started: 2021-12-17 22:53:40 -06:00)


In [7]:
#drops columns that have only NAN values
combined_months.dropna(how='all', axis=1, inplace=True)

time: 31 ms (started: 2021-12-17 22:53:43 -06:00)


In [8]:
combined_months = combined_months.drop(['neighbourhood'], axis=1)

time: 0 ns (started: 2021-12-17 22:53:44 -06:00)


In [ ]:
#combined_months['geom'] = combined_months['latitude'].map(str) + ',' + combined_months['longitude'].map(str)
#combined_months['geom'][0]

In [ ]:
#combined_months.info()

In [ ]:
#combined_months.loc[:,"geom"]

In [9]:
#remember to remove key before saving
api_key = "api_key_goes_here"

time: 0 ns (started: 2021-12-17 22:54:10 -06:00)


In [10]:
geolocator = GoogleV3(api_key="api_key_goes_here")

time: 0 ns (started: 2021-12-17 22:54:13 -06:00)


In [11]:
#location = geolocator.reverse([36.17315,-86.73581], timeout = 10, exactly_one=True)
location = geolocator.reverse("36.17315,-86.73581")

time: 312 ms (started: 2021-12-17 22:54:15 -06:00)


In [12]:
print(location)

1913 Fatherland St, Nashville, TN 37206, USA
time: 0 ns (started: 2021-12-17 22:54:18 -06:00)


In [ ]:
# this does not work with googlev3
#locator = geolocator(user_agent='myGeocoder', timeout=10)
#rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.1)

In [ ]:
tqdm_notebook.pandas()

In [ ]:
#combined_months['address'] = combined_months['geom'].progress_apply(rgeocode)
#combined_months.head()

In [ ]:
#drop duplicates 
#dropduplicates = combined_months.drop_duplicates(subset=['geom'], keep='first')

In [13]:
# create a column to put lat-long into the format google likes - this just makes it easier to call their API
combined_months['latlng'] = combined_months.apply(lambda row: '{},{}'.format(row['latitude'], row['longitude']), axis=1)

time: 937 ms (started: 2021-12-17 22:55:26 -06:00)


In [14]:
combined_months.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,latlng
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"36.17315,-86.73581"
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"36.17826,-86.74162"
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"36.12523,-86.81278"
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"36.16249,-86.58988"
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"36.16102,-86.59207"


time: 16 ms (started: 2021-12-17 22:55:32 -06:00)


In [15]:
dropduplicates = combined_months.drop_duplicates(subset=['latlng'], keep='first')

time: 31 ms (started: 2021-12-17 22:57:46 -06:00)


In [16]:
dropduplicates.shape

(8696, 16)

time: 0 ns (started: 2021-12-17 22:58:00 -06:00)


In [ ]:
#combined_months['latlng'].astype(str)

In [ ]:
#def reversegeopy(latlng):
 #   time.sleep(pause)
  #  geolocator = GoogleV3(api_key)
   # address, _  = geolocator.reverse(latlng, exactly_one=True)
    #return address

#combined_months['address'] = combined_months['latlng'].map(reversegeopy)

In [ ]:
#location = geolocator.reverse("31.2158271,-85.3634326")
#location.raw['display_name']

In [17]:
lats=dropduplicates['latitude'].to_list()
lons=dropduplicates['longitude'].to_list()
# Creating a zip with latitudes and longitudes
coords=list(zip(lats,lons))


time: 0 ns (started: 2021-12-17 22:58:24 -06:00)


In [19]:
len(lats)

8696

time: 15 ms (started: 2021-12-17 22:59:50 -06:00)


In [20]:
len(lons)

8696

time: 0 ns (started: 2021-12-17 23:00:23 -06:00)


In [21]:
full_address=[]
for i in range(len(coords)):
    location = geolocator.reverse(coords[i])
#    address=location.raw['address']
    full_address.append(location)
#Creating dataframe with all the addresses
#addres=pd.DataFrame(data=full_address , columns=['Address'])
#addres
#combined_months['address'] = full_address
full_address

[Location(1913 Fatherland St, Nashville, TN 37206, USA, (36.1729959, -86.7358027, 0.0)),
 Location(205 N 16th St, Nashville, TN 37206, USA, (36.1783179, -86.7414213, 0.0)),
 Location(2803 Natchez Trace, Nashville, TN 37212, USA, (36.1254302, -86.81301719999999, 0.0)),
 Location(4133 Helena Bay Ct, Hermitage, TN 37076, USA, (36.16235229999999, -86.58986709999999, 0.0)),
 Location(172 Lake Chateau Dr, Hermitage, TN 37076, USA, (36.160867, -86.59197800000001, 0.0)),
 Location(4200 Bayside Ln, Hermitage, TN 37076, USA, (36.1631432, -86.5917451, 0.0)),
 Location(401 Montego Cove, Hermitage, TN 37076, USA, (36.16118300000001, -86.58972920000001, 0.0)),
 Location(902 S 12th St, Nashville, TN 37206, USA, (36.1649506, -86.75010329999999, 0.0)),
 Location(622 Philwood Dr, Nashville, TN 37214, USA, (36.1548793, -86.6488483, 0.0)),
 Location(910 Phillips St, Nashville, TN 37208, USA, (36.1722785, -86.7925485, 0.0)),
 Location(1029 Mitchell Rd, Nashville, TN 37206, USA, (36.185555, -86.715533999999

time: 23min 1s (started: 2021-12-17 23:00:51 -06:00)


In [22]:
len(full_address)

8696

time: 0 ns (started: 2021-12-17 23:25:56 -06:00)


In [23]:
dropduplicates['address'] = full_address

time: 16 ms (started: 2021-12-17 23:27:10 -06:00)


C:\Users\YASHAC~1\AppData\Local\Temp/ipykernel_18956/2261414506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropduplicates['address'] = full_address


In [28]:
dropduplicates.head()

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,latlng,address
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"36.17315,-86.73581","(1913 Fatherland St, Nashville, TN 37206, USA,..."
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"36.17826,-86.74162","(205 N 16th St, Nashville, TN 37206, USA, (36...."
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"36.12523,-86.81278","(2803 Natchez Trace, Nashville, TN 37212, USA,..."
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"36.16249,-86.58988","(4133 Helena Bay Ct, Hermitage, TN 37076, USA,..."
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"36.16102,-86.59207","(172 Lake Chateau Dr, Hermitage, TN 37076, USA..."


time: 16 ms (started: 2021-12-17 23:35:21 -06:00)


In [26]:
dropduplicates.shape

(8696, 17)

time: 0 ns (started: 2021-12-17 23:28:17 -06:00)


In [30]:
dropduplicates.head(6)

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,latlng,address
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"36.17315,-86.73581","(1913 Fatherland St, Nashville, TN 37206, USA,..."
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"36.17826,-86.74162","(205 N 16th St, Nashville, TN 37206, USA, (36...."
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"36.12523,-86.81278","(2803 Natchez Trace, Nashville, TN 37212, USA,..."
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"36.16249,-86.58988","(4133 Helena Bay Ct, Hermitage, TN 37076, USA,..."
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"36.16102,-86.59207","(172 Lake Chateau Dr, Hermitage, TN 37076, USA..."
5,319705,"SuperSweetSTUDIO, jacuzzi, monthly",22296,Diana,36.16296,-86.59187,Private room,31,30,48,2020-07-03,0.43,7,356,NaN,"36.16296,-86.59187","(4200 Bayside Ln, Hermitage, TN 37076, USA, (3..."


time: 15 ms (started: 2021-12-17 23:42:28 -06:00)


In [32]:
#dropduplicates = dropduplicates.drop(['latlng'], axis=1)

time: 0 ns (started: 2021-12-17 23:48:36 -06:00)


In [33]:
dropduplicates.head(6)

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,address
0,6422,Nashville Charm,12172,Michele,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"(1913 Fatherland St, Nashville, TN 37206, USA,..."
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"(205 N 16th St, Nashville, TN 37206, USA, (36...."
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"(2803 Natchez Trace, Nashville, TN 37212, USA,..."
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"(4133 Helena Bay Ct, Hermitage, TN 37076, USA,..."
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"(172 Lake Chateau Dr, Hermitage, TN 37076, USA..."
5,319705,"SuperSweetSTUDIO, jacuzzi, monthly",22296,Diana,36.16296,-86.59187,Private room,31,30,48,2020-07-03,0.43,7,356,NaN,"(4200 Bayside Ln, Hermitage, TN 37076, USA, (3..."


time: 15 ms (started: 2021-12-17 23:48:44 -06:00)


In [36]:
dropduplicates.to_csv("fulladdresslist5.csv", sep=',', index=False, header=True)

time: 78 ms (started: 2021-12-18 00:03:22 -06:00)
